In [5]:
import pandas as pd
from rapidfuzz import fuzz
import unicodedata

# === 1. Load CSV ===
csv_file = "20250903_Extrait_Constatations_F2.csv"  # replace with your CSV path
df = pd.read_csv(csv_file, delimiter=';', dtype=str)

# === 2. Clean column names ===
def clean_column(name):
    # Remove leading/trailing spaces and non-breaking spaces
    name = name.strip().replace('\xa0', ' ')
    # Normalize unicode (accents)
    name = unicodedata.normalize('NFC', name)
    return name

df.columns = [clean_column(col) for col in df.columns]
print("Cleaned columns:", df.columns.tolist())

# === 3. Define relevant columns ===
type_col = "Type de constatation Texte"
desc_col = "Description"
comment_col = "Commentaire"

# Check columns exist
for col in [type_col, desc_col, comment_col]:
    if col not in df.columns:
        raise KeyError(f"Column '{col}' not found. Available columns: {df.columns.tolist()}")

# === 4. Drop rows where type text is missing ===
df = df.dropna(subset=[type_col])

# === 5. Fuzzy grouping of similar types ===
unique_types = df[type_col].dropna().unique().tolist()
groups = {}
threshold = 85  # similarity threshold

for text in unique_types:
    found = False
    for key in groups.keys():
        if fuzz.ratio(text, key) >= threshold:
            groups[key].append(text)
            found = True
            break
    if not found:
        groups[text] = [text]

# === 6. Aggregate counts and non-empty descriptions/comments ===
results = []
for group_key, texts in groups.items():
    subset = df[df[type_col].isin(texts)]
    count = len(subset)
    descriptions = subset[desc_col].dropna().unique().tolist()
    comments = subset[comment_col].dropna().unique().tolist()
    results.append({
        "Representative Type": group_key,
        "All Similar Types": texts,
        "Count": count,
        "Descriptions": descriptions,
        "Comments": comments
    })

# === 7. Convert results to DataFrame and display ===
summary_df = pd.DataFrame(results).sort_values(by="Count", ascending=False)
display(summary_df)

# === 8. Save summary to CSV ===
summary_df.to_csv("type_constatation_summary.csv", index=False, sep=';')
print("Summary saved as 'type_constatation_summary.csv'")


Cleaned columns: ['Mandant', 'Nr. OInf', 'Nom OInf', 'Typ OInf', 'Nr. OI', "Type de l'objet de l'inventaire Texte", 'Propriétaire Nom', 'Nr. Tronçon entretien', 'Nr. Axe SRB', 'Nom Axe SRB', 'Nr. Élément', 'Nom Élément', 'Type Élément', 'Dernière Inspection', 'Constatation Type', 'Type de constatation Texte', 'Localisation', 'Description', 'Commentaire']


,Representative Type,All Similar Types,Count,Descriptions,Comments
1,Eclats,[Eclats],5579,"[Abplatzungen von ca. 10 x 10 cm, talseitig un...","[1 Stk. bis 300 cm2, mit korrodierter Bewehrun..."
16,Fissures dues à des déformations empêchées,[Fissures dues à des déformations empêchées],4440,[Vereinzelt im oberen Bereich mit Kalkausschei...,"[Schwindrisse mit Kalkaussinterung , Mehrere S..."
10,Taches de rouille,"[Taches de rouille, Traces de rouille]",3682,"[Korrodiertes Oberflächennahes Eisen, Rostflec...","[Einzelne, kleine , 1 Stk., jetzt Abplatzung, ..."
26,autre constatation,[autre constatation],3031,"[Constatation non trouvée, Garde corps défectu...","[Ouverture du joint de 1.5 à 3 cm environ, Cet..."
2,Lessivage de la chaux,[Lessivage de la chaux],1908,"[Lokal vereinzelte Aussinterungen , im erhebli...","[Undichte Fuge , Anzahl (Stk) = 3, Sickerwasse..."
...,...,...,...,...,...
132,Déformations mineures,[Déformations mineures],1,[],[2. Element oben ca. 2cm vorstehend]
134,Forces d'ancrage excessives,[Forces d'ancrage excessives],1,[],[Ankerverschiebungen oder Anker gerissen]
136,Plaques d'appui déformées,[Plaques d'appui déformées],1,[],[]
139,Couche en téflon trop mince,[Couche en téflon trop mince],1,[],"[0.9mm Fugenspalt, 12. + 18. + 23. Lager nur 0..."


Summary saved as 'type_constatation_summary.csv'
